3. include OpenStreetMap

In [72]:
import geopandas as gpd
import glob
import pandas as pd


# countries

# Pfad zu den Shapefiles (z.B. alle .shp-Dateien in einem Verzeichnis)
shapefile_path = "C:/Users/alexa/Documents/Webapp/GPC-BNDR-ADM-HPR-SAMPLE-WO-SELECTED_SHP/*.shp"

# Lade alle Shapefiles und kombiniere sie in ein GeoDataFrame
gdf_list = []
for shp in glob.glob(shapefile_path):
    gdf = gpd.read_file(shp)
    gdf_list.append(gdf)

# Kombiniere alle GeoDataFrames
all_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

#folium.GeoJson(all_gdf).add_to(map)


# regions = pd.read_csv("C:/Users/alexa/Documents/Webapp/GPC-POST-GEO-SAMPLE-WO-SELECTED/CSV/GPC-REGIONS-GEO-SAMPLE-WO-SELECTED.csv", sep=';')

for index, row in regions.iterrows():
    # Erstelle eine Liste von Koordinaten aus den Spalten 'longitude' und 'latitude'
    coordinates = [(row['latitude'], row['longitude'])]  # Hier wird die Liste mit einem Tuple von Longitude und Latitude erstellt

    # Erstelle ein Polygon
    folium.Polygon(
        locations=coordinates,
        color='blue',  # Farbe des Polygons
        fill=True,
        fill_color='blue',  # Füllfarbe
        fill_opacity=0.6,
        popup=row['region1']  # Name oder weitere Informationen zur Region
    ).add_to(map)



# places = pd.read_csv("C:/Users/alexa/Documents/Webapp/GPC-POST-GEO-SAMPLE-WO-SELECTED/CSV/GPC-PLACES-GEO-SAMPLE-WO-SELECTED.csv", sep=';')

for index, row in places.iterrows():
    # Erstelle eine Liste von Koordinaten aus den Spalten 'longitude' und 'latitude'
    coordinates = [(row['latitude'], row['longitude'])]  # Hier wird die Liste mit einem Tuple von Longitude und Latitude erstellt

    # Erstelle ein Polygon
    folium.Polygon(
        locations=coordinates,
        color='blue',  # Farbe des Polygons
        fill=True,
        fill_color='blue',  # Füllfarbe
        fill_opacity=0.6,
        popup=row['postcode']  # Name oder weitere Informationen zur Region
    ).add_to(map)



import folium

#Creates a map with Mapbox; without tiles and API_key specifiers, a map with OpenStreetMap is created
map = folium.Map(
    location=[52.52, 13.40],
    zoom_start=6,
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/{{z}}/{{x}}/{{y}}?access_token=pk.eyJ1IjoiZWwtZ2lmIiwiYSI6ImNtMXQyYWdsYzAwMGUycXFzdmY2eDFnaWMifQ.yirQoMK5TCdmZZUFUNXxwA',
    attr='Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="https://www.mapbox.com/">Mapbox</a>'
)

folium.GeoJson(r"C:\Users\alexa\Documents\Webapp\geonames-postal-code@public.geojson").add_to(map)

for i, lat in enumerate(ds['latitude'].values[::10]):
    for j, lon in enumerate(ds['longitude'].values[::10]):
        speed = wind_speed[i, j]  # Windgeschwindigkeit an der Position (i, j)
        folium.Marker(
            [lat, lon], 
            popup=f"Windgeschwindigkeit: {speed:.2f} m/s"
        ).add_to(map)

map.save("map.html")

map with wind speed forecasts (works with app.py)
(yet to do: meshgrid produces 2d array with same dimensions in both directions, but we have more longitudinal than latitudinal points --> wind data are stretched in one direction)

In [10]:
import os
import time
from ecmwf.opendata import Client

# Pfad zu den gespeicherten Wetterdaten
data_file_path = "data_europe.grib2"
# Festgelegtes Zeitintervall für Aktualisierungen (in Sekunden)
time_threshold = 10 * 3600  # 10 Stunden in Sekunden
overwrite = 0 # force overwriting of data

# Funktion, um das Alter der Datei zu überprüfen
def is_data_stale(file_path, time_threshold):
    if not os.path.exists(file_path):
        # Datei existiert nicht, Daten müssen abgerufen werden
        return True
    # Alter der Datei ermitteln (in Sekunden seit der letzten Änderung)
    file_age = time.time() - os.path.getmtime(file_path)
    # Überprüfen, ob die Datei älter als der festgelegte Schwellenwert ist
    return file_age > time_threshold

# ECMWF-Client initialisieren
client = Client(
    source="ecmwf",
    model="ifs",
    resol="0p25"
)

# all available steps up to 7 days (168 hours)
step_selection = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
     63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
     123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168]
# all available steps
# step_selection = [
#     0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60,
#     63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120,
#     123, 126, 129, 132, 135, 138, 141, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204,
#     210, 216, 222, 228, 234, 240
# ]

if is_data_stale(data_file_path, time_threshold) or overwrite:
    print("Daten sind veraltet, existieren nicht oder sollen überschrieben werden. Abruf neuer Daten.")
    # Abrufen der API-Daten, da sie entweder fehlen oder älter als 10 Stunden sind
    result = client.retrieve( # retrieve data worldwide, because no area tag available (https://github.com/ecmwf/ecmwf-opendata) in comparison to the paid MARS API (https://confluence.ecmwf.int/display/UDOC/Retrieve)
        type="fc",  
        param=["100v", "100u"],  # U- und V-Komponenten der Windgeschwindigkeit
        target=data_file_path,
        time=0,  # Vorhersagezeit (Modelllauf um 00z)
        step=step_selection
    )
    print("Neue Daten wurden erfolgreich abgerufen und gespeichert.")
else:
    print("Daten sind aktuell und werden verwendet.")


Daten sind veraltet oder existieren nicht. Abruf neuer Daten.


<multiple>:   0%|          | 0.00/137M [00:00<?, ?B/s]

Neue Daten wurden erfolgreich abgerufen und gespeichert.


In [14]:
import os
import numpy as np
import folium
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import xarray as xr
from branca.colormap import linear
import pickle

# Datei laden und als xarray-Dataset öffnen
file_path = "data_europe.grib2"
ds = xr.open_dataset(file_path, engine='cfgrib')

# Definiere eine Farblegende basierend auf typischen Windgeschwindigkeiten
min_wind_speed = 0  # Minimum der Windgeschwindigkeit
max_wind_speed = 30  # Angenommenes Maximum der Windgeschwindigkeit
colormap = linear.Paired_06.scale(min_wind_speed, max_wind_speed)  # Farbskala von 0 bis 30 m/s, https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html, this sweeping approach saves from ressource-hungry operations

# Iteriere über alle Zeitschritte (steps) im Dataset
layers = []
for step_index, step_value in enumerate(ds['step'].values):

    # Extrahiere die Windkomponenten für den aktuellen Zeitschritt
    u_component = ds['u100'].isel(step=step_index).values
    v_component = ds['v100'].isel(step=step_index).values
    lats = ds['latitude'].values
    lons = ds['longitude'].values

    # Bereich für Europa definieren
    lat_min, lat_max = 45, 55 # Breitengradbereich für Europa: 35, 72
    lon_min, lon_max = -5, 5 # Längengradbereich für Europa: -25, 45

    # Filtere die Breitengrade (latitude) und Längengrade (longitude) auf den gewünschten Bereich für Europa
    lat_indices = np.where((lats >= lat_min) & (lats <= lat_max))[0]
    lon_indices = np.where((lons >= lon_min) & (lons <= lon_max))[0]

    # Trunkiere die Längen- und Breitengrade auf den europäischen Bereich
    lats_europe = lats[lat_indices]
    lons_europe = lons[lon_indices]
 
    # Extrahiere die Windgeschwindigkeitsdaten für Europa
    u_component_europe = u_component[np.ix_(lat_indices, lon_indices)]
    v_component_europe = v_component[np.ix_(lat_indices, lon_indices)]

    # Berechne die Windgeschwindigkeit für den europäischen Bereich
    wind_speed = np.sqrt(u_component_europe**2 + v_component_europe**2)

    # Interpolationsgitter für Europa festlegen
    grid_lat = np.linspace(lats_europe.min(), lats_europe.max(), 2*len(lats_europe))  # Anpassbar nach gewünschter Auflösung
    grid_lon = np.linspace(lons_europe.min(), lons_europe.max(), 2*len(lons_europe))

    # Interpolation der Windgeschwindigkeit auf das definierte Gitter
    points_data = np.array([(lon, lat) for lon in lons_europe for lat in lats_europe])
    grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)  # Erstelle ein Meshgrid für die Gitterkoordinaten
    grid_wind_speed = griddata(points_data, wind_speed.ravel(), 
                                    (grid_lon, grid_lat), method='cubic')

    overlay_image = np.empty(grid_wind_speed.shape + (4,), dtype=np.uint8)  # Erstelle ein leeres Bildarray
    for i in range(grid_wind_speed.shape[0]):
        for j in range(grid_wind_speed.shape[1]):
            # Wende die Colormap an und stelle sicher, dass der RGBA-Wert korrekt ist
            hex_color = colormap(grid_wind_speed[i, j])  # Wende Colormap an
            
            # Umwandlung von hexadezimalem RGBA in Integer-Werte
            r = int(hex_color[1:3], 16)  # Rot-Wert
            g = int(hex_color[3:5], 16)  # Grün-Wert
            b = int(hex_color[5:7], 16)  # Blau-Wert
            a = int(hex_color[7:9], 16)  # Alpha-Wert

            overlay_image[i, j] = np.array([r, g, b, a], dtype=np.uint8)  # Speichern der RGBA-Werte

    # Speichern des Layers für die Shiny-App
    layer_name = f"Zeitschritt {step_index}"  # Verwende nur den Index
    layers.append((layer_name, overlay_image))

# Speichere die Layer-Daten und die lat/lon-Informationen in einer Datei (z.B. pickle oder JSON)
data_to_save = {
    "layers": layers,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "step_selection": step_selection
}

with open("wind_layers.pkl", "wb") as f:
    pickle.dump(data_to_save, f)


Ignoring index file 'data_europe.grib2.5b7b6.idx' older than GRIB file


map with postal codes

In [58]:
import requests
import geopandas as gpd
import folium
from folium import GeoJson

# API URL and parameters
base_api_url = "https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/geonames-postal-code@public/records"
limit = 20 # API pagination
total_count = 999999  # Total number of records expected
all_geojson_data = []  # List to store all geojson data

# Fetch all pages of data
for offset in range(0, total_count, limit):
    api_url = f"{base_api_url}?limit={limit}&offset={offset}"
    
    # Send API request
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the data in JSON format
        data = response.json()
        
        # Check if 'results' exists in the data
        if 'results' in data:
            # Extract GeoJSON data from JSON
            geojson_data = data['results']
            all_geojson_data.extend(geojson_data)  # Add the current results
        else:
            print("Key 'results' not found in the data.")
            break  # End loop if no results are found
    else:
        print(f"Request failed with status code: {response.status_code}, offset: {offset}")
        break  # End loop if an error occurs

# Convert GeoJSON data to a GeoDataFrame
features = [
    feature['geo_shape'] for feature in all_geojson_data if 'geo_shape' in feature
]

# Create the GeoDataFrame and set the CRS
gdf2 = gpd.GeoDataFrame.from_features(features)
gdf2.set_geometry('coordinates', inplace=True)  # Set 'coordinates' column as geometry
gdf2.set_crs(epsg=4326, inplace=True)  # Set CRS to EPSG:4326





# Erstelle eine Folium-Karte
m = folium.Map(location=[20, 0], zoom_start=2)  # Setze den Mittelpunkt und den Zoom der Karte

# Füge die GeoDataFrame-Daten zur Karte hinzu
GeoJson(gdf2).add_to(m)


# Speichere die Karte als HTML-Datei oder zeige sie an
m.save('world_boundaries_map.html')


Request failed with status code: 400, offset: 10000


ValueError: Unknown column coordinates

map with boundaries and WPP (HeatMap and MaskerCluster)

In [3]:
import requests
import geopandas as gpd
import folium
from folium import GeoJson
from folium.plugins import HeatMap, MarkerCluster
import numpy as np
import pandas as pd



# API URL und Parameter für die Ländergrenzen
base_api_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/world-administrative-boundaries/records"
limit = 20  # API-Paginierung
total_count = 256  # Gesamtzahl der erwarteten Datensätze
all_geojson_data = []  # Liste zur Speicherung aller GeoJSON-Daten

# Fetch all pages of data
for offset in range(0, total_count, limit):
    api_url = f"{base_api_url}?limit={limit}&offset={offset}"
    
    # API Anfrage senden
    response = requests.get(api_url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Die Daten im JSON-Format extrahieren
        boundary_data = response.json()
        
        # Überprüfen, ob 'results' in den Daten vorhanden ist
        if 'results' in boundary_data:
            # GeoJSON-Daten aus dem JSON extrahieren
            geojson_data = boundary_data['results']
            all_geojson_data.extend(geojson_data)  # Füge die aktuellen Ergebnisse hinzu
        else:
            print("Key 'results' nicht in den Daten gefunden.")
            break  # Beende die Schleife, wenn keine Ergebnisse gefunden werden
    else:
        print(f"Fehler bei der Anfrage: {response.status_code}")
        break  # Beende die Schleife, wenn ein Fehler auftritt

# Konvertieren der GeoJSON-Daten in ein GeoDataFrame
features = []
for feature in all_geojson_data:
    if 'geo_shape' in feature:
        geom = feature['geo_shape']['geometry']  # Behalte die Geometrie
        properties = {
            'name': feature.get('name', None)  # Füge den Namen hinzu
        }
        features.append({
            'geometry': geom,
            'properties': properties
        })

# Erstelle das GeoDataFrame und setze das CRS
gdf1 = gpd.GeoDataFrame.from_features(features)
gdf1.set_crs(epsg=4326, inplace=True)  # Setze das CRS auf EPSG:4326




# Windkraftanlagen-Daten abrufen
url = "https://overpass-api.de/api/interpreter?data=[out:json][timeout:99999];node['power'='generator']['generator:source'='wind'](49,9,50,10);out body;" # Europe: (34.8,-30,71.6,60)
response = requests.get(url)

if response.status_code == 200:
    wind_data = response.json()
else:
    print("Fehler beim Abrufen der Windkraftdaten:", response.status_code)
    wind_data = None

# Karte erstellen
m = folium.Map(location=[50.68, 6.72], zoom_start=12)  # Erftstadt-Mitte

# Markercluster-Layer erstellen
marker_layer = folium.FeatureGroup(name="Marker Cluster")
marker_cluster = MarkerCluster().add_to(marker_layer)

# Heatmap-Layer erstellen
heatmap_layer = folium.FeatureGroup(name="Heatmap")

# Kapazitäten nach Land summieren
if wind_data:
    # Initialisiere ein Dictionary zur Speicherung der Kapazitäten nach Ländern
    capacities_by_country = {name: 0 for name in gdf1['name']}  # Verwende die Namen aus gdf1
    heat_data = []
    
    for element in wind_data["elements"]:
        try:
            # Extrahiere die Kapazität aus den tags
            if 'generator:output:electricity' in element['tags']:
                capacity_value = element['tags']['generator:output:electricity']
                # Prüfen und umrechnen je nach Einheit
                if 'kW' in capacity_value:
                    capacity = float(capacity_value.replace('kW', '').strip())
                elif 'MW' in capacity_value:
                    capacity = float(capacity_value.replace('MW', '').strip()) * 1000  # Umrechnung in kW
                elif capacity_value.lower() == 'yes':
                    capacity = 0  # Wenn keine Kapazität angegeben ist
                else:
                    capacity = float(capacity_value)  # Konvertiere direkt zu float, falls keine Einheit
            else:
                capacity = 0  # Wenn keine Kapazität angegeben ist

        except ValueError:
            capacity = 0  # Fehler beim Konvertieren

        # Überprüfen, in welchem Land sich die Windkraftanlage befindet
        point = gpd.points_from_xy([element['lon']], [element['lat']])
        for idx, country in gdf1.iterrows():
            if country['geometry'].contains(point[0]):
                capacities_by_country[country['name']] += capacity

        # Daten zur Heatmap hinzufügen
        heat_data.append([element["lat"], element["lon"], capacity])

        # Marker mit Tooltip zur Markergruppe hinzufügen
        folium.Marker(
            location=[element['lat'], element['lon']],
            tooltip=(
                f'Kapazität: {capacity} kW\n'
                f'Baujahr: {element["tags"].get("construction", "n/a")}\n'
                f'Betreiber: {element["tags"].get("operator", "n/a")}'
            )
        ).add_to(marker_cluster)

    # Heatmap zum Heatmap-Layer hinzufügen
    HeatMap(data=heat_data, radius=15, max_zoom=13).add_to(heatmap_layer)

    # Heatmap und Marker-Layer zur Karte hinzufügen
    heatmap_layer.add_to(m)
    marker_layer.add_to(m)

    # Füge die Ländergrenzen zur Karte hinzu
    GeoJson(gdf1.geometry).add_to(m)

    folium.Choropleth(
        geo_data=gdf1,
        name='choropleth',
        data=capacities_by_country,  # Dein Dictionary mit den Kapazitäten
        columns=['name', 'capacity'],  # Hier muss "capacity" vorhanden sein
        key_on='feature.properties.name',  # Schlüssel zur Zuordnung der Länder
        fill_color='YlGn',  # Farbskala
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Windkraftkapazität (kW)'
    ).add_to(m)

    # Füge Layer Controls hinzu
    folium.LayerControl().add_to(m)

    # Karte speichern (optional)
    m.save("wind_energy.html") # just for own visualisation, not used by app.py

    # Überprüfen, ob Features vorhanden sind
    print(f"{len(wind_data['elements'])} Windkraftdaten erfolgreich zur Karte hinzugefügt")
else:
    print("Keine Windkraftdaten verfügbar.")


420 Windkraftdaten erfolgreich zur Karte hinzugefügt
